参考：[隠れマルコフモデルをざっくり理解したい](https://qiita.com/s-kojima1227/items/36e2468570c9bc62a481)

In [1]:
import numpy as np
import numpy.linalg as LA
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

In [8]:
# モデルの自由度
K = 2
D = 6
# 遷移確率
A_transition = np.array([
    [0.9, 0.1],
    [0.1, 0.9]
])
# 潜在変数で条件づけられた観測変数の出力確率
B_emission = np.array([
    [1./6.]*6,
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]
])
# 混合係数
pi = np.array(
    [1., 0.]
)

# 繰り返し上限
T = 100

# 観測データ列
x = np.array([
    4, 4, 5, 4, 2, 3, 3, 6, 4, 5, 5, 3, 4, 4, 1, 4, 5, 3, 6, 5
])

In [3]:
def forward(x, A_transition, B_emission, pi):
    N = x.shape[0]
    # K = A_transition.shape[0]
    # D = B_emission.shape[1]
    alpha = np.zeros((N, K))

    alpha[0, :] = pi * B_emission[:, x[0]]

    for n in range(1, N):
        for k in range(K):
            alpha[n, k] = B_emission[k, x[n]] * (alpha[n-1, :] @ A_transition[:, k])

    return alpha

In [4]:
def backward(x, A_transition, B_emission, pi):
    N = x.shape[0]
    # K = A_transition.shape[0]
    # D = B_emission.shape[1]
    beta = np.zeros((N, K))

    beta[N-1] = np.ones((K))

    for n in reversed(range(0, N-1)):
        for k in range(K):
            beta[n, k] = beta[n+1, :] * B_emission[:, x[n]] @ A_transition[k, :]

    return beta

In [6]:
def E_step(x, A_transition, B_emission, pi):
    N = x.shape[0]
    # K = A_transition.shape[0]
    # D = B_emission.shape[1]
    alpha = forward(x, A_transition, B_emission, pi)
    beta = backward(x, A_transition, B_emission, pi)
    prob_X = np.sum(alpha[N-1, :])

    gamma = np.zeros((N, K))
    xi = np.zeros((N-1, K, K))

    for n in range(N):
        gamma[n, :] = alpha[n, :] * beta[n, :] / prob_X
        if n == N-1:
            break
        for i in range(K):
            for j in range(K):
                xi[n, i, j] = alpha[n, i] * A_transition[i, j] * B_emission[j, x[n+1]] * beta[n+1, j]
        xi[n, :, :] /= prob_X

    return gamma, xi

In [7]:
def M_step(x, gamma, xi):
    N = x.shape[0]
    A_new = np.zeros((K, K))
    B_new = np.zeros((K, D))
    pi_new = gamma[0, :]

    for i in range(K):
        for j in range(K):
            A_new[i, j] = np.sum(xi[0:N-1, i, j])
        A_new[i, :] /= np.sum(gamma[0:N-1, i])

        for d in range(D):
            for n in range(N):
                B_new[i, d] += (x[n] == d) * gamma[n, i]
        B_new[i, :] /= np.sum(gamma[:, i])

    return A_new, B_new, pi_new

In [9]:
def EM(x, A_transition, B_emission, pi):
    N = x.shape[0]
    for t in range(T):
        gamma, xi = E_step(x, A_transition, B_emission, pi)
        A_new, B_new, pi_new = M_step(x, gamma, xi)

        A_transition = A_new
        B_emission = B_new
        pi = pi_new

    return A_transition, B_emission, pi

In [ ]:
def viterbi(x, A_transition, B_emission, pi):
    N = x.shape[0]

    